In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler
from statsmodels.datasets import co2
from statsmodels.tsa.seasonal import STL

In [ ]:
src_url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
df = pd.read_csv(src_url, index_col='Date')
df.index = pd.to_datetime(df.index)
df = df[df.index >= datetime.strptime('2010-01-01', '%Y-%m-%d')]
df = df.resample('W').sum()

In [ ]:
df

In [ ]:
df['trend_Wind'] = df['Wind'].rolling(52, center=True).mean().rolling(2, center=True).mean()
df['detrended_Wind'] = df['Wind'] - df['trend_Wind']

df['seasonal_Wind'] = df['detrended_Wind'].groupby(df.index.isocalendar().week).transform(np.mean)
df['seasonal_Wind'] = df['seasonal_Wind'] - df['seasonal_Wind'].mean()

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(18,21), sharey=True, sharex=True)

axs[0].plot(df['Wind'])
axs[1].plot(df['trend_Wind'])
axs[2].plot(df['seasonal_Wind'])
axs[3].plot(df['Wind'] - df['trend_Wind'] - df['seasonal_Wind'])

In [ ]:
stl = STL(df['Wind'], seasonal=7, trend=105)

In [ ]:
res = stl.fit()

In [ ]:
fig = res.plot()

In [ ]:
res.resid.hist()

In [ ]:
df = co2.load(as_pandas=True).data.resample('M').mean().fillna(10_000)

In [ ]:
stl = STL(df, seasonal=17)
res = stl.fit()

In [ ]:
res.plot()
plt.show()

In [ ]:
stl.period